# **MODELO S**
Sem data augmentation

## **Importações e Definições de Seed**

Primeiro, importamos as bibliotecas necessárias e definimos a seed para garantir a reprodutibilidade dos resultados.

In [4]:
import optuna
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import random


SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

## **Definição dos Diretórios do Dataset e Tamanho da Imagem**

Definimos os diretórios dos conjuntos de dados e o tamanho das imagens.
Juntamos as imagens de treino todas na mesma pastas para simplificar o carregamento.

In [5]:
train_dir = '../Data/train'
validation_dir = '../Data/validation5'
test_dir = '../Data/test'
IMG_SIZE = 32
BATCH_SIZE = 32

### Funções para Visualizaçao de Resultados

In [6]:
def plot_training_history(history, test_number, trial_number):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.savefig(f'../plots/s{test_number}/trial_{trial_number}_plot.png')

    plt.close()
    return


def predict_with_best_model(model, dataset):
    y_true = []
    y_pred = []

    for x, y in dataset:
        y_true.extend(tf.argmax(y, axis=1))
        y_pred.extend(tf.argmax(model.predict(x), axis=1))

    return y_true, y_pred

# Função para plotar matriz de confusão
def plot_confusion_matrix(y_true, y_pred, class_names, test_number,trial_number):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title('Confusion Matrix')
    plt.savefig(f'../plots/s{test_number}/trial_{trial_number}_confusion_matrix.png')

    plt.close()
    return

## **Carregamento dos Conjuntos de Dados**

In [11]:
train_dataset = image_dataset_from_directory(
    train_dir,
    label_mode='categorical',
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    seed=SEED
)

validation_dataset = image_dataset_from_directory(
    validation_dir,
    label_mode='categorical',
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    seed=SEED
)

test_dataset = image_dataset_from_directory(
    test_dir,
    label_mode='categorical',
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    seed=SEED
)

Found 40000 files belonging to 10 classes.
Found 10000 files belonging to 10 classes.
Found 10000 files belonging to 10 classes.


## **Normalização dos Conjuntos de Dados**

Normalizamos os conjuntos de dados para garantir que os valores dos pixels estejam na faixa de 0 a 1 o que são mais adequados ao modelo de treino.

In [14]:
normalization_layer = layers.Rescaling(1./255)

train_dataset = train_dataset.map(
    lambda x, y: (normalization_layer(x), y)
)

validation_dataset = validation_dataset.map(
    lambda x, y: (normalization_layer(x), y)
)

test_dataset = test_dataset.map(
    lambda x, y: (normalization_layer(x), y)
)

# **Introdução à Otimização de Hiperparâmetros com Optuna**

A otimização de hiperparâmetros é crucial para desenvolver modelos de machine learning eficazes. Com o Optuna, uma biblioteca poderosa de otimização automática, podemos explorar eficientemente o espaço de hiperparâmetros para encontrar as melhores configurações. 
Integrar o Optuna ao nosso projeto automatiza o ajuste fino dos hiperparâmetros da nossa CNN, garantindo um modelo mais preciso e robusto. Não só acelera a experimentação, mas também maximiza o desempenho do modelo, contribuindo significativamente para o sucesso do projeto.

### **Função para Criar o Modelo de CNN**

Definimos a função `create_model`, que cria um modelo de rede neural convolucional com base nos hiperparâmetros fornecidos.

`Dropout` inserido com taxa de 0.3 após as camadas convolucionais e 0.5 após as camadas fully connected para regularizar o modelo, prevenindo overfitting ao desligar aleatoriamente unidades durante o treinamento.

Obs: Na terceira Camada optamos por não fazer o `MaxPooling2D` pois os filtros ficariam muito pequenos.

In [8]:
def create_model(
    num_filters_1,
    num_filters_2,
    num_filters_3,
    optimizer_name,
    learning_rate,
    momentum,
    img_size
):
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(img_size, img_size, 3)))

    # Primeira camada convolucional
    model.add(layers.Conv2D(num_filters_1, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.3))

    # Segunda camada convolucional
    model.add(layers.Conv2D(num_filters_2, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.3))

    # Terceira camada convolucional
    model.add(layers.Conv2D(num_filters_3, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))

    # Camadas flatten e dense
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation='softmax'))

    # Compilar o modelo com o otimizador especificado
    if optimizer_name == 'SGD':
        optimizer = optimizers.SGD(learning_rate=learning_rate, momentum=momentum)
    else:
        optimizer = getattr(optimizers, optimizer_name)(learning_rate=learning_rate)

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Exemplo da estrutura Base

In [7]:
num_filters_1 = 32
num_filters_2 = 32
num_filters_3 = 32
optimizer_name = 'Adam'
learning_rate = 1e-3

model = models.Sequential()
model.add(layers.InputLayer(input_shape=(IMG_SIZE, IMG_SIZE, 3)))

# Primeira camada convolucional
model.add(layers.Conv2D(num_filters_1, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.3))

# Segunda camada convolucional
model.add(layers.Conv2D(num_filters_2, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.3))

# Terceira camada convolucional
model.add(layers.Conv2D(num_filters_3, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))

# Camadas flatten e dense
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

optimizer = getattr(optimizers, optimizer_name)(learning_rate=learning_rate)

model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy'])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 30, 30, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 13, 13, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 13, 13, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 6, 6, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 4, 4, 32)       │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 4, 4, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 4, 4, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 287,562 (1.10 MB)

 Trainable params: 287,370 (1.10 MB)

 Non-trainable params: 192 (768.00 B)

### **Função Objetivo para Otimização**

A função `objective` é utilizada na otimização dos hiperparâmetros do modelo utilizando o Optuna. Nesta função, sugerimos diferentes valores para ajustar os parâmetros da rede neural convolucional (CNN), visando melhorar seu desempenho.

Nesta configuração, exploramos diferentes números de filtros para as camadas convolucionais e opções de otimizadores como Adam ou SGD. Além disso, ajustamos a taxa de aprendizado de forma logarítmica para uma busca mais eficiente no espaço de hiperparâmetros. Essa abordagem nos permite encontrar combinações ideais que maximizem a precisão e a robustez do nosso modelo de CNN.

- ***num_filters_1***, ***num_filters_2***, ***num_filters_3***

  - Determinam o número de filtros nas camadas convolucionais da rede neural. Com opções de [32, 64], você está testando dois conjuntos de valores para cada camada convolucional. Um valor mais baixo como 32 pode ser adequado para capturar características mais simples, enquanto 64 pode capturar características mais complexas. Testar ambos permite avaliar qual configuração resulta em melhores resultados de desempenho e generalização para sua tarefa específica.

- ***optimizer_name***

  - Define o otimizador a ser usado durante o treinamento da rede neural. As opções 'Adam' e 'SGD' oferecem escolhas entre um otimizador adaptativo comumente usado (Adam) e um otimizador estocástico de gradiente descendente (SGD). O que influencia a velocidade e a precisão da convergência durante o treinamento.

- ***learning_rate***

  - Usar uma distribuição loguniforme entre 1e-4 e 1e-3 permite testar uma ampla gama de valores de taxa de aprendizado. Valores menores podem ajudar na convergência, enquanto valores maiores podem acelerar o treinamento inicial, mas podem ser menos estáveis.

- ***Callbacks***
  - `ModelCheckpoint`
      - Guarda o melhor modelo de cada trail.

  - `EarlyStopping`
      - Reduz a taxa de aprendizado do modelo quando a métrica 'val_loss' para de melhorar, ajudando a garantir que o treinamento seja estável e eficaz.

  - `ReduceLROnPlateau`
    - Interrompe o treinamento quando a métrica 'val_loss' para de melhorar por um número especificado de épocas (***patience***), restaurando os melhores pesos obtidos durante o treinamento.

Os parâmetros foram selecionados para equilibrar eficiência computacional, capacidade de aprendizado e generalização do modelo, seguindo práticas comuns em redes neurais convolucionais para garantir desempenho aceitavel em tarefas de classificação de imagens.

In [10]:
def objective(trial):
    # Estabelecer os valores sugeridos para cada hiperparâmetro
    num_filters_1 = trial.suggest_categorical('num_filters_1', [32, 64])
    num_filters_2 = trial.suggest_categorical('num_filters_2', [32, 64])
    num_filters_3 = trial.suggest_categorical('num_filters_3', [32, 64])
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD'])
    learning_rate = trial.suggest_categorical('learning_rate', [1e-3, 1e-4])
    momentum = 0.9
    
    model = create_model(
        num_filters_1, 
        num_filters_2, 
        num_filters_3,
        optimizer_name, 
        learning_rate,
        momentum,
        IMG_SIZE
    )

    # Definir callbacks, incluindo ModelCheckpoint para salvar o melhor modelo
    model_dir = '../models/s1/'
    trial_number = str(trial.number)
    model_path = model_dir + 'model_trial_' + trial_number + '_CPbest.keras'
    callbacks = [
        ModelCheckpoint(
            filepath=model_path,
            save_best_only=True,
            monitor='val_loss',
            verbose=1
        ),
        EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        ),
        #   Uma vez que o modelo esta a optimizar baseado em accuracy aqui 
        # colocamos em relação ao loss para ter mais parametros a ajudar 
        # a aproximar do melhor modelo.
        ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.2, 
            patience=3, 
            min_lr=1e-5
        )
    ]

    # Treinar o modelo
    history = model.fit(
        train_dataset,
        epochs=10,
        validation_data=validation_dataset,
        callbacks=callbacks,
        verbose=0
    )

    plot_training_history(history, '1', trial_number)
    
    return history.history['val_loss'][-1]

### **Otimização do Modelo**

numero de trails =  2 (num_filters_1) × 2 (num_filters_2) × 2 (num_filters_3) × 2 (otimizadores) × 2 (learning_rate)  = 32

In [12]:
study = optuna.create_study(
    direction='minimize', 
    sampler=optuna.samplers.TPESampler(seed=SEED)
)

study.optimize(objective, n_trials=32)

[I 2024-06-19 00:29:44,169] A new study created in memory with name: no-name-f2b0a884-9144-4be3-886b-4950bd39f407



Epoch 1: val_loss improved from inf to 1.92279, saving model to ../models/model_trial_0_CPbest.keras

Epoch 2: val_loss improved from 1.92279 to 1.91952, saving model to ../models/model_trial_0_CPbest.keras

Epoch 3: val_loss improved from 1.91952 to 1.64900, saving model to ../models/model_trial_0_CPbest.keras

Epoch 4: val_loss improved from 1.64900 to 1.61996, saving model to ../models/model_trial_0_CPbest.keras

Epoch 5: val_loss improved from 1.61996 to 1.55388, saving model to ../models/model_trial_0_CPbest.keras

Epoch 6: val_loss improved from 1.55388 to 1.51339, saving model to ../models/model_trial_0_CPbest.keras

Epoch 7: val_loss improved from 1.51339 to 1.49111, saving model to ../models/model_trial_0_CPbest.keras

Epoch 8: val_loss improved from 1.49111 to 1.49050, saving model to ../models/model_trial_0_CPbest.keras

Epoch 9: val_loss improved from 1.49050 to 1.46203, saving model to ../models/model_trial_0_CPbest.keras

Epoch 10: val_loss did not improve from 1.46203


[I 2024-06-19 00:32:52,791] Trial 0 finished with value: 1.4938578605651855 and parameters: {'num_filters_1': 64, 'num_filters_2': 32, 'num_filters_3': 32, 'optimizer': 'SGD', 'learning_rate': 0.0001}. Best is trial 0 with value: 1.4938578605651855.



Epoch 1: val_loss improved from inf to 1.45938, saving model to ../models/model_trial_1_CPbest.keras

Epoch 2: val_loss improved from 1.45938 to 1.36350, saving model to ../models/model_trial_1_CPbest.keras

Epoch 3: val_loss did not improve from 1.36350

Epoch 4: val_loss did not improve from 1.36350

Epoch 5: val_loss improved from 1.36350 to 1.22301, saving model to ../models/model_trial_1_CPbest.keras

Epoch 6: val_loss did not improve from 1.22301

Epoch 7: val_loss improved from 1.22301 to 1.10295, saving model to ../models/model_trial_1_CPbest.keras

Epoch 8: val_loss did not improve from 1.10295

Epoch 9: val_loss improved from 1.10295 to 1.09825, saving model to ../models/model_trial_1_CPbest.keras

Epoch 10: val_loss did not improve from 1.09825


[I 2024-06-19 00:35:59,995] Trial 1 finished with value: 1.1705812215805054 and parameters: {'num_filters_1': 64, 'num_filters_2': 32, 'num_filters_3': 64, 'optimizer': 'SGD', 'learning_rate': 0.001}. Best is trial 1 with value: 1.1705812215805054.


...

[I 2024-06-19 02:01:40,388] Trial 29 finished with value: 1.5980913639068604 and parameters: {'num_filters_1': 64, 'num_filters_2': 32, 'num_filters_3': 32, 'optimizer': 'SGD', 'learning_rate': 0.0001}. Best is trial 8 with value: 0.8072530031204224.



Epoch 1: val_loss improved from inf to 1.51509, saving model to ../models/model_trial_30_CPbest.keras

Epoch 2: val_loss improved from 1.51509 to 1.43308, saving model to ../models/model_trial_30_CPbest.keras

Epoch 3: val_loss improved from 1.43308 to 1.14382, saving model to ../models/model_trial_30_CPbest.keras

Epoch 4: val_loss did not improve from 1.14382

Epoch 5: val_loss did not improve from 1.14382

Epoch 6: val_loss improved from 1.14382 to 1.11738, saving model to ../models/model_trial_30_CPbest.keras

Epoch 7: val_loss improved from 1.11738 to 1.05754, saving model to ../models/model_trial_30_CPbest.keras

Epoch 8: val_loss improved from 1.05754 to 1.01211, saving model to ../models/model_trial_30_CPbest.keras

Epoch 9: val_loss improved from 1.01211 to 0.90329, saving model to ../models/model_trial_30_CPbest.keras

Epoch 10: val_loss did not improve from 0.90329


[I 2024-06-19 02:04:46,224] Trial 30 finished with value: 0.9787349700927734 and parameters: {'num_filters_1': 32, 'num_filters_2': 32, 'num_filters_3': 32, 'optimizer': 'Adam', 'learning_rate': 0.001}. Best is trial 8 with value: 0.8072530031204224.



Epoch 1: val_loss improved from inf to 1.38603, saving model to ../models/model_trial_31_CPbest.keras

Epoch 2: val_loss did not improve from 1.38603

Epoch 3: val_loss improved from 1.38603 to 1.15574, saving model to ../models/model_trial_31_CPbest.keras

Epoch 4: val_loss improved from 1.15574 to 1.08317, saving model to ../models/model_trial_31_CPbest.keras

Epoch 5: val_loss improved from 1.08317 to 0.99886, saving model to ../models/model_trial_31_CPbest.keras

Epoch 6: val_loss did not improve from 0.99886

Epoch 7: val_loss improved from 0.99886 to 0.99254, saving model to ../models/model_trial_31_CPbest.keras

Epoch 8: val_loss improved from 0.99254 to 0.93782, saving model to ../models/model_trial_31_CPbest.keras

Epoch 9: val_loss improved from 0.93782 to 0.90749, saving model to ../models/model_trial_31_CPbest.keras

Epoch 10: val_loss improved from 0.90749 to 0.82351, saving model to ../models/model_trial_31_CPbest.keras


[I 2024-06-19 02:07:52,525] Trial 31 finished with value: 0.8235142827033997 and parameters: {'num_filters_1': 32, 'num_filters_2': 32, 'num_filters_3': 32, 'optimizer': 'Adam', 'learning_rate': 0.001}. Best is trial 8 with value: 0.8072530031204224.


- ## Trial **0** 
  - Perda de Validação: 1.4938578605651855 

  - {Layer 1: 64, Layer 2: 32, Layer 3: 32, Optimizer: SGD, Learning Rate: 0.0001}

  
![](../plots/s1/trial_0_plot.png) 

<br>

- ## Trial **1** 
  
  - Perda de Validação: 1.1705812215805054

  - {Layer 1: 64, Layer 2: 32, Layer 3: 64, Optimizer: SGD, Learning Rate: 0.001}

![](../plots/s1/trial_1_plot.png) 

<br>

- ## Trial **2** 

  - Perda de Validação: 1.1758157014846802

  - {Layer 1: 32, Layer 2: 64, Layer 3: 64, Optimizer: SGD, Learning Rate: 0.001}

![](../plots/s1/trial_2_plot.png) 

<br>

- ## Trial **3** 

  - Perda de Validação: 0.8584625124931335

  - {Layer 1: 32, Layer 2: 64, Layer 3: 32, Optimizer: Adam, Learning Rate: 0.001}
  
![](../plots/s1/trial_3_plot.png) 

<br>

- ## Trial **4** 
  - Perda de Validação: 1.4459023475646973

  - {Layer 1: 64, Layer 2: 64, Layer 3: 64, Optimizer: SGD, Learning Rate: 0.001}

![](../plots/s1/trial_4_plot.png) 

<br>

- ## Trial **5** 

  - Perda de Validação: 1.547646403312683

  - {Layer 1: 32, Layer 2: 32, Layer 3: 64, Optimizer: SGD, Learning Rate: 0.0001}
  
![](../plots/s1/trial_5_plot.png) 

<br>

- ## Trial **6** 

  - Perda de Validação: 1.409365177154541

  - {Layer 1: 32, Layer 2: 32, Layer 3: 64, Optimizer: SGD, Learning Rate: 0.0001}
  
![](../plots/s1/trial_6_plot.png) 

<br>

- ## Trial **7**

  - Perda de Validação: 0.8536289930343628

  - {Layer 1: 32, Layer 2: 64, Layer 3: 64, Optimizer: Adam, Learning Rate: 0.001}
  
![](../plots/s1/trial_7_plot.png) 

<br>

- ## Trial **8**
  
  - Perda de Validação: 0.8072530031204224

  - {Layer 1: 32, Layer 2: 32, Layer 3: 64, Optimizer: Adam, Learning Rate: 0.001}
  
![](../plots/s1/trial_8_plot.png) 

<br>

- ## Trial **9** 
  - Perda de Validação: 1.1882990598678589

  - {Layer 1: 64, Layer 2: 32, Layer 3: 32, Optimizer: Adam, Learning Rate: 0.0001}

![](../plots/s1/trial_9_plot.png) 

<br>

- ## Trial **10** 
  - Perda de Validação: 0.8523767590522766

  - {Layer 1: 32, Layer 2: 32, Layer 3: 32, Optimizer: Adam, Learning Rate: 0.001}

![](../plots/s1/trial_10_plot.png) 

<br>

- ## Trial **11** 
  - Perda de Validação: 0.8630514740943909

  - {Layer 1: 32, Layer 2: 32, Layer 3: 32, Optimizer: Adam, Learning Rate: 0.001}

![](../plots/s1/trial_11_plot.png) 

<br>

- ## Trial **12** 
  - Perda de Validação: 0.9670686721801758

  - {Layer 1: 32, Layer 2: 32, Layer 3: 32, Optimizer: Adam, Learning Rate: 0.001}

![](../plots/s1/trial_12_plot.png) 

<br>

- ## Trial **13** 
  - Perda de Validação: 1.0514543056488037

  - {Layer 1: 32, Layer 2: 32, Layer 3: 32, Optimizer: Adam, Learning Rate: 0.001}

![](../plots/s1/trial_13_plot.png) 

<br>

- ## Trial **14** 
  - Perda de Validação: 0.8961184620857239

  - {Layer 1: 32, Layer 2: 32, Layer 3: 64, Optimizer: Adam, Learning Rate: 0.001}

![](../plots/s1/trial_14_plot.png) 

<br>

- ## Trial **15** 
  - Perda de Validação: 0.8445688486099243

  - {Layer 1: 32, Layer 2: 32, Layer 3: 32, Optimizer: Adam, Learning Rate: 0.001}

![](../plots/s1/trial_15_plot.png) 

<br>

- ## Trial **16** 
  - Perda de Validação: 0.9276623725891113

  - {Layer 1: 32, Layer 2: 32, Layer 3: 64, Optimizer: Adam, Learning Rate: 0.001}

![](../plots/s1/trial_16_plot.png) 

<br>

- ## Trial **17**

  - Perda de Validação: 0.8921107649803162

  - {Layer 1: 32, Layer 2: 32, Layer 3: 32, Optimizer: Adam, Learning Rate: 0.001}

![](../plots/s1/trial_17_plot.png) 

<br>

- ## Trial **18**

  - Perda de Validação: 1.149288296699524

  - {Layer 1: 64, Layer 2: 64, Layer 3: 64, Optimizer: Adam, Learning Rate: 0.0001}

![](../plots/s1/trial_18_plot.png) 

<br>

- ## Trial **19**

  - Perda de Validação: 0.8507242202758789

  - {Layer 1: 32, Layer 2: 32, Layer 3: 32, Optimizer: Adam, Learning Rate: 0.001}

![](../plots/s1/trial_19_plot.png) 

<br>

- ## Trial **20**

  - Perda de Validação: 0.8496494889259338

  - {Layer 1: 32, Layer 2: 32, Layer 3: 32, Optimizer: Adam, Learning Rate: 0.001}

![](../plots/s1/trial_20_plot.png) 

<br>

- ## Trial **21**

  - Perda de Validação: 0.8548123836517334

  - {Layer 1: 32, Layer 2: 32, Layer 3: 32, Optimizer: Adam, Learning Rate: 0.001}

![](../plots/s1/trial_21_plot.png) 

<br>

- ## Trial **22**

  - Perda de Validação: 0.9142076373100281

  - {Layer 1: 32, Layer 2: 32, Layer 3: 32, Optimizer: Adam, Learning Rate: 0.001}

![](../plots/s1/trial_22_plot.png) 

<br>

- ## Trial **23**

  - Perda de Validação: 0.8966420888900757

  - {Layer 1: 32, Layer 2: 32, Layer 3: 32, Optimizer: Adam, Learning Rate: 0.001}

![](../plots/s1/trial_23_plot.png) 

<br>

- ## Trial **24**

  - Perda de Validação: 1.1329811811447144

  - {Layer 1: 32, Layer 2: 32, Layer 3: 32, Optimizer: Adam, Learning Rate: 0.001}

![](../plots/s1/trial_24_plot.png) 

<br>

- ## Trial **25**

  - Perda de Validação: 0.8683971762657166

  - {Layer 1: 32, Layer 2: 32, Layer 3: 32, Optimizer: Adam, Learning Rate: 0.001}

![](../plots/s1/trial_25_plot.png) 

<br>

- ## Trial **26**

  - Perda de Validação: 1.1191442012786865

  - {Layer 1: 64, Layer 2: 64, Layer 3: 64, Optimizer: Adam, Learning Rate: 0.0001}

![](../plots/s1/trial_26_plot.png) 

<br>

- ## Trial **27**

  - Perda de Validação: 0.926841676235199

  - {Layer 1: 32, Layer 2: 32, Layer 3: 32, Optimizer: Adam, Learning Rate: 0.001}

![](../plots/s1/trial_27_plot.png) 

<br>

- ## Trial **28**

  - Perda de Validação: 0.8615323305130005

  - {Layer 1: 32, Layer 2: 32, Layer 3: 64, Optimizer: Adam, Learning Rate: 0.001}

![](../plots/s1/trial_28_plot.png) 

<br>

- ## Trial **29** 

  - Perda de Validação: 1.5980913639068604

  - {Layer 1: 64, Layer 2: 32, Layer 3: 32, Optimizer: SGD, Learning Rate: 0.0001}

![](../plots/s1/trial_29_plot.png) 

<br>

- ## Trial **30** 

  - Perda de Validação: 0.9787349700927734

  - {Layer 1: 32, Layer 2: 32, Layer 3: 32, Optimizer: Adam, Learning Rate: 0.001}

![](../plots/s1/trial_30_plot.png) 

<br>

- ## Trial **31** 

  - Perda de Validação: 0.8235142827033997

  - {Layer 1: 32, Layer 2: 32, Layer 3: 32, Optimizer: Adam, Learning Rate: 0.001}

![](../plots/s1/trial_31_plot.png) 
  
<br>

# **Visualização dos Resultados**

### Melhor modelo **8** 

In [24]:
# Carregar o melhor modelo encontrado
best_model_path ='../models/s1/model_trial_8_CPbest.keras'

best_model = models.load_model(best_model_path)

In [18]:
# Avaliar o melhor modelo no conjunto de teste
test_loss, test_acc = best_model.evaluate(test_dataset)
print(f'Precião no conjunto de Teste: {test_acc:.4f}')
print(f'Perda no conjunto de Teste: {test_loss:.4f}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.7189 - loss: 0.8086
Precião no conjunto de Teste: 0.7124
Perda no conjunto de Teste: 0.8212


In [25]:
# Matriz de confusão

# Fazer previsões no conjunto de teste
y_true, y_pred = predict_with_best_model(best_model, test_dataset)

class_names = ['Avião', 'Automovel', 'Passaro', 'Gato', 'Viado',
                'Cão', 'Sapo', 'Cavalo', 'Barco', 'Camião']
plot_confusion_matrix(y_true, y_pred, class_names, '1', '8')

I0000 00:00:1719074295.574267   18933 service.cc:145] XLA service 0x7f2f840019f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1719074295.574317   18933 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2024-06-22 17:38:15.601839: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


I0000 00:00:1719074296.545963   18933 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

2024-06-22 17:38:38.874421: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


# Matriz de Confusão do modelo **8**

Na Matriz de confusão poderos reparar que dentro de categorias que são animais e maquinas existem mais falhas. 

Isto ocorre devido à semelhança entre eles por exemplo as cores, linhas mais retas, etc. 

![](../plots/s1/trial_8_confusion_matrix.png)

# Selecionar Modelos Interessantes

Dos modelos criados anteriormente, embora tenham sido repedidos alguns parametros e tenham sido feitas poucas épocas podemos tirar algumas conclusões.
Os modelos que mostraram resultados mais interessantes foram o 3, 7, 8 e 15.

- **Filtros por Layer**
    - Regra geral apenas os modelos com 32 filtros no primeiro layer tiveram bom resultados.
    - Os valores utilizados nos layers 2 e 3 não parecem influenciar muito no resultado, mas o melhor fui 32, 64 respetivamente então vamos guardar esses parametros para o modelo final.

- **Optimizador e Learning Rate**
    - O optimizador `Adaptive Moment Estimation` foi o que gerou melhores resultados. De seguida iremos experimentar os mais valores de learning rate para os modelos.

Vamos também aumentar o número de epocas uma vez que os graficos não mostram sinais de overfitting para estes modelos

In [9]:
def objective(trial):
    num_filters_1 = 32
    num_filters_2 = 32
    num_filters_3 = 64
    optimizer_name = 'Adam'
    learning_rate = trial.suggest_categorical('learning_rate', [1e-2, 5e-3, 3e-4, 1e-5, 5e-5])
    momentum = None
    
    model = create_model(
        num_filters_1, 
        num_filters_2, 
        num_filters_3,
        optimizer_name, 
        learning_rate,
        momentum,
        IMG_SIZE
    )
    
    model_dir = '../models/s2/'
    trial_number = str(trial.number)
    model_path = model_dir + 'model_trial_' + trial_number + '_CPbest.keras'

    callbacks = [
        ModelCheckpoint(
            filepath=model_path,
            save_best_only=True,
            monitor='val_loss',
            verbose=1
        ),
        EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
        ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.2, 
            patience=3, 
            min_lr=1e-5
        )
    ]

    history = model.fit(
        train_dataset,
        epochs=20,
        validation_data=validation_dataset,
        callbacks=callbacks,
        verbose=0
    )

    plot_training_history(history, '2', trial_number)
    
    return history.history['val_loss'][-1]

In [10]:
study = optuna.create_study(
    direction='minimize', 
    sampler=optuna.samplers.TPESampler(seed=SEED)
)

study.optimize(objective, n_trials=5)

[I 2024-06-20 16:04:28,467] A new study created in memory with name: no-name-b0b25ca8-a019-41ae-8e60-f26e36a59753
/home/patinhooh/miniconda3/envs/ia/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
I0000 00:00:1718895870.759888   21209 service.cc:145] XLA service 0x7f7d3c001de0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1718895870.759994   21209 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2024-06-20 16:04:30.921842: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-20 16:04:31.437061: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1718895877.699153   21209 device_compiler.h:188] Compiled cluster using XLA!  


Epoch 1: val_loss improved from inf to 1.46992, saving model to ../models/s2/model_trial_0_CPbest.keras

Epoch 2: val_loss did not improve from 1.46992

Epoch 3: val_loss did not improve from 1.46992

Epoch 4: val_loss did not improve from 1.46992

Epoch 5: val_loss improved from 1.46992 to 0.96520, saving model to ../models/s2/model_trial_0_CPbest.keras

Epoch 6: val_loss improved from 0.96520 to 0.89891, saving model to ../models/s2/model_trial_0_CPbest.keras

Epoch 7: val_loss did not improve from 0.89891

Epoch 8: val_loss did not improve from 0.89891

Epoch 9: val_loss did not improve from 0.89891

Epoch 10: val_loss improved from 0.89891 to 0.80748, saving model to ../models/s2/model_trial_0_CPbest.keras

Epoch 11: val_loss improved from 0.80748 to 0.78331, saving model to ../models/s2/model_trial_0_CPbest.keras

Epoch 12: val_loss did not improve from 0.78331

Epoch 13: val_loss did not improve from 0.78331

Epoch 14: val_loss improved from 0.78331 to 0.78065, saving model to .

[I 2024-06-20 16:11:29,313] Trial 0 finished with value: 0.75468909740448 and parameters: {'learning_rate': 0.005}. Best is trial 0 with value: 0.75468909740448.



Epoch 1: val_loss improved from inf to 1.48253, saving model to ../models/s2/model_trial_1_CPbest.keras

Epoch 2: val_loss improved from 1.48253 to 1.36841, saving model to ../models/s2/model_trial_1_CPbest.keras

Epoch 3: val_loss did not improve from 1.36841

Epoch 4: val_loss improved from 1.36841 to 1.18898, saving model to ../models/s2/model_trial_1_CPbest.keras

Epoch 5: val_loss did not improve from 1.18898

Epoch 6: val_loss improved from 1.18898 to 1.18084, saving model to ../models/s2/model_trial_1_CPbest.keras

Epoch 7: val_loss improved from 1.18084 to 0.95129, saving model to ../models/s2/model_trial_1_CPbest.keras

Epoch 8: val_loss improved from 0.95129 to 0.91447, saving model to ../models/s2/model_trial_1_CPbest.keras

Epoch 9: val_loss improved from 0.91447 to 0.86661, saving model to ../models/s2/model_trial_1_CPbest.keras

Epoch 10: val_loss improved from 0.86661 to 0.84770, saving model to ../models/s2/model_trial_1_CPbest.keras

Epoch 11: val_loss did not improve

[I 2024-06-20 16:17:54,819] Trial 1 finished with value: 0.8625155091285706 and parameters: {'learning_rate': 0.0003}. Best is trial 0 with value: 0.75468909740448.


...

[I 2024-06-20 16:24:21,964] Trial 2 finished with value: 0.7907170653343201 and parameters: {'learning_rate': 0.005}. Best is trial 0 with value: 0.75468909740448.



Epoch 1: val_loss improved from inf to 1.62718, saving model to ../models/s2/model_trial_3_CPbest.keras

Epoch 2: val_loss improved from 1.62718 to 1.35559, saving model to ../models/s2/model_trial_3_CPbest.keras

Epoch 3: val_loss improved from 1.35559 to 1.22014, saving model to ../models/s2/model_trial_3_CPbest.keras

Epoch 4: val_loss did not improve from 1.22014

Epoch 5: val_loss improved from 1.22014 to 1.07628, saving model to ../models/s2/model_trial_3_CPbest.keras

Epoch 6: val_loss improved from 1.07628 to 1.01566, saving model to ../models/s2/model_trial_3_CPbest.keras

Epoch 7: val_loss did not improve from 1.01566

Epoch 8: val_loss improved from 1.01566 to 0.95026, saving model to ../models/s2/model_trial_3_CPbest.keras

Epoch 9: val_loss did not improve from 0.95026

Epoch 10: val_loss improved from 0.95026 to 0.92810, saving model to ../models/s2/model_trial_3_CPbest.keras

Epoch 11: val_loss did not improve from 0.92810

Epoch 12: val_loss improved from 0.92810 to 0.

[I 2024-06-20 16:30:44,040] Trial 3 finished with value: 0.7082935571670532 and parameters: {'learning_rate': 0.0003}. Best is trial 3 with value: 0.7082935571670532.



Epoch 1: val_loss improved from inf to 1.55734, saving model to ../models/s2/model_trial_4_CPbest.keras

Epoch 2: val_loss did not improve from 1.55734

Epoch 3: val_loss did not improve from 1.55734

Epoch 4: val_loss did not improve from 1.55734

Epoch 5: val_loss improved from 1.55734 to 1.22165, saving model to ../models/s2/model_trial_4_CPbest.keras

Epoch 6: val_loss improved from 1.22165 to 1.13440, saving model to ../models/s2/model_trial_4_CPbest.keras

Epoch 7: val_loss did not improve from 1.13440

Epoch 8: val_loss improved from 1.13440 to 1.09052, saving model to ../models/s2/model_trial_4_CPbest.keras

Epoch 9: val_loss improved from 1.09052 to 1.04598, saving model to ../models/s2/model_trial_4_CPbest.keras

Epoch 10: val_loss did not improve from 1.04598

Epoch 11: val_loss did not improve from 1.04598

Epoch 12: val_loss improved from 1.04598 to 0.96659, saving model to ../models/s2/model_trial_4_CPbest.keras

Epoch 13: val_loss did not improve from 0.96659

Epoch 14:

[I 2024-06-20 16:37:05,702] Trial 4 finished with value: 0.8736234307289124 and parameters: {'learning_rate': 0.01}. Best is trial 3 with value: 0.7082935571670532.


- ## Trial **0** 
  
  - Loss Validation: 0.75468909740448

  - Learning Rate: 0.005

![](../plots/s2/trial_0_plot.png) 

<br>

- ## Trial **1** 
  
  - Loss Validation: 0.8625155091285706

  - Learning Rate: 0.0003

![](../plots/s2/trial_1_plot.png) 

<br>

- ## Trial **2** 
  
  - Loss Validation: 0.7907170653343201

  - Learning Rate: 0.005

![](../plots/s2/trial_2_plot.png) 

<br>

- ## Trial **3** 
  
  - Loss Validation: 0.7082935571670532

  - Learning Rate: 0.0003

![](../plots/s2/trial_3_plot.png) 

<br>

- ## Trial **4** 
  
  - Loss Validation: 0.8736234307289124

  - Learning Rate: 0.01

![](../plots/s2/trial_4_plot.png) 

<br>

### Verificar resultados dos testes de learning rate

In [12]:

for i in range(5):
    model_path =f'../models/s2/model_trial_{i}_CPbest.keras'
    print("\nModel: ",i,)
    model = models.load_model(model_path)
    # Avaliar o melhor modelo no conjunto de teste
    test_loss, test_acc = model.evaluate(test_dataset)
    print(f'\tPrecião no conjunto de Teste: {test_acc:.4f}')
    print(f'\tPerda no conjunto de Teste:   {test_loss:.4f}')


Model:  0
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.7336 - loss: 0.7636
	Precião no conjunto de Teste: 0.7304
	Perda no conjunto de Teste:   0.7758

Model:  1
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.7505 - loss: 0.7117
	Precião no conjunto de Teste: 0.7476
	Perda no conjunto de Teste:   0.7260

Model:  2
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.7295 - loss: 0.7831
	Precião no conjunto de Teste: 0.7175
	Perda no conjunto de Teste:   0.8123

Model:  3
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.7604 - loss: 0.7014
	Precião no conjunto de Teste: 0.7559
	Perda no conjunto de Teste:   0.7164

Model:  4
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.7165 - loss: 0.8225
	Precião no conjunto de Teste: 0.7061
	Perda no conjunto de Teste:   0.8439


# Tentar treinar mais o modelo **3**

In [15]:
model = models.load_model('../models/s2/model_trial_3_CPbest.keras')
model_path = '../models/s2/model_trial_3_1_CPbest.keras'

callbacks = [
    ModelCheckpoint(
        filepath=model_path,
        save_best_only=True,
        monitor='val_loss',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.2, 
        patience=3, 
        min_lr=1e-5
    )
]

history = model.fit(
    train_dataset,
    epochs=20,
    validation_data=validation_dataset,
    callbacks=callbacks,
    verbose=0
)

plot_training_history(history, '2', '3_1')


Epoch 1: val_loss improved from inf to 0.76635, saving model to ../models/s2/model_trial_3_1_CPbest.keras

Epoch 2: val_loss did not improve from 0.76635

Epoch 3: val_loss improved from 0.76635 to 0.71462, saving model to ../models/s2/model_trial_3_1_CPbest.keras

Epoch 4: val_loss did not improve from 0.71462

Epoch 5: val_loss did not improve from 0.71462

Epoch 6: val_loss improved from 0.71462 to 0.70207, saving model to ../models/s2/model_trial_3_1_CPbest.keras

Epoch 7: val_loss improved from 0.70207 to 0.70048, saving model to ../models/s2/model_trial_3_1_CPbest.keras

Epoch 8: val_loss did not improve from 0.70048

Epoch 9: val_loss did not improve from 0.70048

Epoch 10: val_loss improved from 0.70048 to 0.69179, saving model to ../models/s2/model_trial_3_1_CPbest.keras

Epoch 11: val_loss did not improve from 0.69179

Epoch 12: val_loss improved from 0.69179 to 0.67346, saving model to ../models/s2/model_trial_3_1_CPbest.keras

Epoch 13: val_loss did not improve from 0.6734

- ## Model **3** 
  
  - 40 epocas no total
  - Perda de Validação: 0.63378

![](../plots/s2/trial_3_1_plot.png) 

<br>

# Avaliação do Modelo 3 depois de mais épocas

In [16]:
# Avaliar o melhor modelo no conjunto de teste
test_loss, test_acc = model.evaluate(test_dataset)
print(f'\tPrecião no conjunto de Teste: {test_acc:.4f}')
print(f'\tPerda no conjunto de Teste:   {test_loss:.4f}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.7826 - loss: 0.6413
	Precião no conjunto de Teste: 0.7795
	Perda no conjunto de Teste:   0.6486


Como se pode ver o modelo ainda está melhorar ainda que ligeiramente. Então vamos tentar treinar durante mais algumas épocas com os callbacks um pouco diferentes.

- `EarlyStopping`
    - Vamos aumentar um pouco mais a *patience* para que ele tenha mais espaço para poder progredir.

- `ReduceLROnPlateau`
    - Reduzir *factor* para o learning rate reduzir um pouco mais rápidamente, e ajustar um pouco a *patience* para o aplicar.

E como a perda de validação já se encontra em níveis bons vamos experimentar monitorar a precisão desta vez.

In [19]:
model = models.load_model('../models/s2/model_trial_3_1_CPbest.keras')
model_path = '../models/s2/model_trial_3_2_CPbest.keras'
callbacks = [
    ModelCheckpoint(
        filepath=model_path,
        save_best_only=True,
        monitor='val_accuracy',
        mode='max',
        verbose=0
    ),
    EarlyStopping(
        monitor='val_accuracy',
        mode='max',
        patience=10
    ),
    ReduceLROnPlateau(
        monitor='val_accuracy', 
        mode='max',
        factor=0.1,  
        patience=5,  
        min_lr=1e-5
    )
]

history = model.fit(
    train_dataset,
    epochs=50,  
    validation_data=validation_dataset,
    callbacks=callbacks,
    verbose=1
)

plot_training_history(history, '2', '3_2')

Epoch 1/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 25s 16ms/step - accuracy: 0.7747 - loss: 0.6315 - val_accuracy: 0.7704 - val_loss: 0.6810 - learning_rate: 6.0000e-05
Epoch 2/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.7753 - loss: 0.6338 - val_accuracy: 0.7776 - val_loss: 0.6477 - learning_rate: 6.0000e-05
Epoch 3/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.7791 - loss: 0.6221 - val_accuracy: 0.7795 - val_loss: 0.6481 - learning_rate: 6.0000e-05
Epoch 4/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.7831 - loss: 0.6155 - val_accuracy: 0.7805 - val_loss: 0.6394 - learning_rate: 6.0000e-05
Epoch 5/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.7789 - loss: 0.6204 - val_accuracy: 0.7842 - val_loss: 0.6284 - learning_rate: 6.0000e-05
Epoch 6/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - accuracy: 0.7830 - loss: 0.6129 - val_accuracy: 0.7796 - val_loss: 0.6438 - learning_rate: 6.0000e-05
Epoch 7/50
1250/1250 ━━━━━━━━━━━━━━━━━━━

# Modelo 3
  - 81 épocas no total
  - Precisão de Validação: 0.7908

![](../plots/s2/trial_3_2_plot.png) 

<br>

# Desempenho Geral do Modelo

In [28]:
# Avaliar o melhor modelo no conjunto de teste
model = models.load_model('../models/s3/model_trial_3_2_DA_CPbest.keras')
y_test, pred = predict_with_best_model(model, test_dataset)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 389ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━

2024-06-22 17:41:02.381085: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [29]:
test_loss, test_acc = model.evaluate(test_dataset)
print(f'\tPrecião no conjunto de Teste: {test_acc:.4f}')
print(f'\tPerda no conjunto de Teste:   {test_loss:.4f}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.7433 - loss: 0.7408
	Precião no conjunto de Teste: 0.7350
	Perda no conjunto de Teste:   0.7643


- Precisão no conjunto de Teste: 73.5%
- Perda no conjunto de Teste: 0.7643

O modelo apresenta um desempenho razoável, mas ainda há espaço para melhorias. 

In [33]:
acc = accuracy_score(y_true =  y_test, y_pred = pred)
print(f'Accuracy : {np.round(acc*100,2)}%')

print("\nPrecision:")   
precision = precision_score(y_true =  y_test, y_pred = pred, average='macro')
print(f'\tMacro:    {np.round(precision*100,2)}%')

precision = precision_score(y_true =  y_test, y_pred = pred, average='weighted')
print(f'\tWeighted: {np.round(precision*100,2)}%')

print("\nRecall:")
recall = recall_score(y_true =  y_test, y_pred = pred, average='macro')
print(f'\tMacro:    {np.round(recall*100,2)}%')

recall = recall_score(y_true =  y_test, y_pred = pred, average='weighted')
print(f'\tWeighted: {np.round(recall*100,2)}%')
print()

print("\nF1-score:")
f1 = f1_score(y_true =  y_test, y_pred = pred, average='macro')
print(f'\tMacro:    {np.round(f1*100,2)}%')

f1 = f1_score(y_true =  y_test, y_pred = pred, average='weighted')
print(f'\tWeighted: {np.round(f1*100,2)}%')

Accuracy : 73.5%

Precision:
	Macro:    74.13%
	Weighted: 74.13%

Recall:
	Macro:    73.5%
	Weighted: 73.5%


F1-score:
	Macro:    72.67%
	Weighted: 72.67%


Resultado Precisão:
- Macro: 74.13%
- Ponderada: 74.13%

A precisão macro e weighted são iguais, indicando que as classes são balanceadas. A precisão de aproximadamente 74% mostra que, quando o modelo prevê uma classe como positiva, ele está correto em 74% dos casos.

<br>

Resultado Revocação:
- Macro: 73.5%
- Ponderada: 73.5%

As métricas de precisão e recall são bastante equilibradas, o que é positivo. Ambas as versões macro e ponderada são similares, indicando que o modelo está tratando as classes de forma relativamente uniforme.

<br>

Resultado F1-Score:
- Macro: 72.67%
- Ponderada: 72.67%

O F1-score é ligeiramente mais baixo que a precisão e o recall indica que pode haver um pequeno desequilíbrio entre precisão e recall. Ajustes adicionais, como balanceamento de classes ou alteração da função de perda, poderiam ajudar a melhorar esta métrica.

In [32]:
class_names = ['Avião', 'Automovel', 'Passaro', 'Gato', 'Viado',
                'Cão', 'Sapo', 'Cavalo', 'Barco', 'Camião']
plot_confusion_matrix(y_test, pred, class_names, '2', '3_2')

![](../plots/s2/trial_3_2_confusion_matrix.png)

- Neste resultado podemos ver que os quadrados entre as categoria animal e maquina já se encontram mais diluidos.

# Considerações finais

Como o modelo começou a apresentar alguns sinais de ovefitting vamos ficar por aqui com este modelo S sem data augmentation.
Utilizaremos o que descobrimos aqui no ajuste do modelo com data augmentation.